Imports Librares

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import pytz
from datetime import datetime

Load dataset

In [ ]:
df = pd.read_csv(r'C:\Users\vishal\Desktop\Dataset\Play Store Data.csv')

In [ ]:
df.head()

Time Window Function

In [ ]:
# Time check function
def is_visible_time(start_hour, end_hour):
    ist = pytz.timezone('Asia/Kolkata')
    now = datetime.now(ist)
    return start_hour <= now.hour < end_hour

# Only run between 3PM to 5PM IST
if is_visible_time(15, 17):
    # Clean Installs
    df['Installs'] = df['Installs'].str.replace('[+,]', '', regex=True)
    df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')

    # Clean Size
    def convert_size(size):
        if isinstance(size, str):
            if 'M' in size:
                return float(size.replace('M', '').strip())
            elif 'k' in size:
                return float(size.replace('k', '').strip()) / 1024
        return None
    df['Size_MB'] = df['Size'].apply(convert_size)

    # Clean Last Updated
    df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')

    # Filter data
    filtered_df = df[
        (df['Rating'] >= 4.0) &
        (df['Size_MB'] >= 10) &
        (df['Last Updated'].dt.month == 1)
    ]

    # Aggregate
    grouped = filtered_df.groupby('Category').agg({
        'Rating': 'mean',
        'Reviews': 'sum',
        'Installs': 'sum'
    }).reset_index()

    # Top 10 by Installs
    top10 = grouped.sort_values(by='Installs', ascending=False).head(10)

    # Plot grouped bar chart
    fig = go.Figure(data=[
        go.Bar(name='Average Rating', x=top10['Category'], y=top10['Rating']),
        go.Bar(name='Total Reviews', x=top10['Category'], y=top10['Reviews'])
    ])

    fig.update_layout(
        barmode='group',
        title='Top 10 App Categories by Installs (Filtered)',
        xaxis_title='Category',
        yaxis_title='Value',
        legend_title='Metric',
        template='plotly_dark'
    )

    fig.show()
else:
    print("⏳ This chart is only visible between 3 PM and 5 PM IST.")